In [1]:
import os
import cv2
from cvloop import cvloop
import face_recognition
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import matrix
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [2]:
def analize_video(video):
    vidcap= cv2.VideoCapture(video)
    print("Frames Rate: %f frames/second"%vidcap.get(cv2.CAP_PROP_FPS))
    print("Frame Count: %d frames"%int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)))
    vidcap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    duration=int(vidcap.get(cv2.CAP_PROP_POS_MSEC)/1000)
    print("Video Duration: %d seconds"%duration)
    width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
    height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print("width,height :",int(width),"X",int(height))

def get_frames(video_path):  
    cap= cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()  
        if ret==True:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames 

def check_identity(all_frames,known_frame,frame_rate=30):
    known_encoding=face_recognition.face_encodings(cv2.imread(known_frame))[0]
    unknown_encodings=[face_recognition.face_encodings(frame)[0] for frame in all_frames]
    results = face_recognition.compare_faces(unknown_encodings, known_encoding)
    person_detected=['Verkha' if x else 'Unknown' for x in results]
    sub_lists = [person_detected[i: i+frame_rate] for i in range(0, len(person_detected), frame_rate)]
    person_count_per_sec=[sub_list.count('Unknown') for sub_list in sub_lists]
    
    return [1 if i>=25 else 0 for i in person_count_per_sec]
    

In [3]:
analize_video('/home/ubuntu/videos/interview_videos/face_identify.mp4') 

Frames Rate: 30.105092 frames/second
Frame Count: 491 frames
Video Duration: 16 seconds
width,height : 1280 X 720


In [4]:
all_frames=get_frames('/home/ubuntu/videos/interview_videos/face_identify.mp4')

In [5]:
another_person=check_identity(all_frames,'/home/ubuntu/videos/interview_videos/verkha.jpg')

In [7]:
another_person

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

# multi-face detection using face_recognition

In [8]:
def multiface_detect(frames, frame_rate=30):
    num_faces = [len(face_recognition.face_locations(i)) for i in frames]
    faces_per_second = [num_faces[i: i+frame_rate] for i in range(0, len(num_faces), frame_rate)]
    person_count = [len([x for x in per_second if x > 1]) for per_second in faces_per_second]
    return [1 if i > 25 else 0 for i in person_count]

In [9]:
analize_video('/home/ubuntu/videos/interview_videos/two_persons.mp4')

Frames Rate: 30.108317 frames/second
Frame Count: 410 frames
Video Duration: 13 seconds
width,height : 1280 X 720


In [10]:
frames=get_frames('/home/ubuntu/videos/interview_videos/two_persons.mp4')

In [11]:
another_person=multiface_detect(frames)

In [12]:
another_person

[1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]

# video_writing

In [ ]:
def face_detect(frame):
    face_locations = face_recognition.face_locations(frame) 
    cnt=str(len(face_locations))
    for face_location in face_locations: ## Not necessary
        (t,r,b,l)=face_location
        frame = cv2.rectangle(frame, (l, t), (r, b), (0,0,255), 2)
        cv2.putText(frame, 'persons detected: '+cnt, (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA,thickness=2)        
    return frame

def write_video(frames):   
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')        #frame rate         
    out = cv2.VideoWriter('Two_faces.avi',fourcc, 30.0, (width, height))
    for frame in frames:
        out.write(frame)        

In [ ]:
frames=get_frames('/home/ubuntu/videos/interview_videos/two_persons.mp4')

In [ ]:
processed_frame=[face_detect(frame) for frame in frames]
write_video(processed_frame)

# multi-object detection using cvlib

In [ ]:
import cvlib as cv

In [ ]:
def get_all_frames(video_path):  
    cap= cv2.VideoCapture(video_path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()  
        if ret==True:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames 

def detect_faces(frame):
    face_locations, label, conf = cv.detect_common_objects(frame) 
    cnt=str(len(face_locations))    
    for face_location in face_locations:
        (sx,sy,ex,ey)=face_location
        frame = cv2.rectangle(frame, (sx, sy), (ex, ey), (0,0,255), 2)
        cv2.putText(frame, 'persons detected: '+cnt, (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), lineType=cv2.LINE_AA,thickness=2)       
    return frame

def write_video(frames):   
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')        #frame rate         
    out = cv2.VideoWriter('multi_obj_detection.avi',fourcc, 30.0, (width, height))
    for frame in frames:
        out.write(frame)

In [ ]:
frames=get_all_frames('/home/ubuntu/videos/interview_videos/two_persons.mp4')

In [ ]:
frames_with_detected_faces=[detect_faces(frame) for frame in frames]

In [ ]:
len(frames_with_detected_faces)

In [ ]:
write_video(frames_with_detected_faces)